# RESNET MODEL FROM SCRATCH

**Import statements**

In [ ]:
import os
import random
import numpy as np
from PIL import Image, ImageEnhance, ImageOps
import cv2
from sklearn.utils import shuffle
from sklearn.metrics import confusion_matrix, accuracy_score
from tensorflow.keras import Sequential
from tensorflow.keras.applications import MobileNetV2
from tensorflow.random import set_seed
from tensorflow.keras.backend import clear_session
from tensorflow.keras.layers import Input, Conv2D, ELU, BatchNormalization,concatenate, \
                                    Add, GlobalAveragePooling2D, Flatten, Dense, MaxPooling2D
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.models import Model, load_model
import matplotlib
from matplotlib import pyplot as plt

**Set up random seed**

In [ ]:
radom_seed = 0
random.seed(radom_seed)
numpy_seed = 0
np.random.seed(numpy_seed)
tensorflow_seed = 0
set_seed(tensorflow_seed)

**Set up config variables**

In [ ]:
IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS, N_CLASSES = 224,224,3,9
BASE_DIR = os.path.abspath(os.path.dirname("__file__"))
INPUT_DIR = os.path.join(BASE_DIR, "data", "NA_Fish_Dataset")
SAVE_DIRECTORY = os.path.join(BASE_DIR, "data", "numpy_data")
MODEL_DIR = os.path.join(BASE_DIR, "model_files")
MODEL_PATH = os.path.join(MODEL_DIR, "model_mobilenetv2.h5")

## 1. Visualize Image Data

In [ ]:
classes_to_labels = dict(zip(list(os.walk(INPUT_DIR))[0][1], [i for i in range(10)]))
labels_to_classes = dict(zip([i for i in range(10)], list(os.walk(INPUT_DIR))[0][1]))

In [ ]:
X_train = np.load(os.path.join(SAVE_DIRECTORY, "X_train.npy"))
X_valid = np.load(os.path.join(SAVE_DIRECTORY, "X_valid.npy"))
Y_train = np.load(os.path.join(SAVE_DIRECTORY, "Y_train.npy"))
Y_valid = np.load(os.path.join(SAVE_DIRECTORY, "Y_valid.npy"))

In [ ]:
matplotlib.rcParams['figure.figsize'] = (30.0, 20.0)
for i in range(20):
    plt.subplot(4,5,i+1)
    plt.title(labels_to_classes[Y_valid[i]])
    plt.imshow(X_valid[i])
    plt.axis('off')
plt.show()

In [ ]:
matplotlib.rcParams['figure.figsize'] = (30.0, 20.0)
for i in range(20):
    plt.subplot(4,5,i+1)
    plt.title(labels_to_classes[Y_train[i]])
    plt.imshow(X_train[i])
    plt.axis('off')
plt.show()

## 2. Create Mobilenet Based Model

**Load mobilenet version 2 to be used as base model**

In [ ]:
clear_session()
mobilenet_v2 = MobileNetV2(include_top = False,input_shape = (IMG_HEIGHT,IMG_WIDTH,3), pooling = 'avg')

**Create and compile model**

In [ ]:
learning_rate = (1e-3)*1
model = \
Sequential((Input(shape=(IMG_HEIGHT,IMG_WIDTH,IMG_CHANNELS)),
            mobilenet_v2,
            Dense(128, activation='elu'),
            Dense(N_CLASSES, activation='softmax')))
for layer in model.layers:
    layer.trainable = True
model.compile(optimizer=Adam(learning_rate=learning_rate),
                  loss=SparseCategoricalCrossentropy(),
                  metrics=['accuracy'])
model.summary()

## 3. Train Model

In [ ]:
def train_model(model, epochs, batch_size, checkpoint_cb, 
                X_train, Y_train, X_valid, Y_valid):
    history = model.fit(X_train, Y_train, epochs=epochs, batch_size = batch_size,
                        validation_data = (X_valid, Y_valid), 
                        callbacks=[checkpoint_cb])
    return model, history

In [ ]:
epochs = 300
batch_size = 100
checkpoint_cb = ModelCheckpoint(MODEL_PATH, monitor = 'val_accuracy', 
                                save_freq = "epoch", save_best_only=True, 
                                mode = "max")
model, history = train_model(model, epochs, batch_size, checkpoint_cb, 
                             X_train, Y_train, X_valid, Y_valid)

## 4. Access performance of model

**Load loss and metrics over epochs**

In [ ]:
history_dict = history.history
loss = history_dict['loss']
val_loss = history_dict['val_loss']
acc = history_dict['accuracy']
val_acc = history_dict['val_accuracy']

**Visualize loss vs epoch graph**

In [ ]:
plt.figure(figsize=(16, 8), dpi= 80, facecolor='w', edgecolor='k')
epochs = range(1, len(loss) + 1)
plt.plot(epochs, loss,  label='Training Loss', linewidth = 10.0)
plt.plot(epochs, val_loss,  label='Validation Loss', linewidth = 3)
plt.title('Loss vs Epochs', fontsize = 25)
plt.xlabel('Epochs', fontsize = 15)
plt.ylabel('Loss', fontsize = 15)
plt.legend()
plt.grid(True)
plt.show()

**Visualize accuracy vs epoch graph**

In [ ]:
plt.figure(figsize=(16, 8), dpi= 80, facecolor='w', edgecolor='k')
epochs = range(1, len(acc) + 1)
plt.plot(epochs, acc,  label='Training Accuracy', linewidth = 10.0)
plt.plot(epochs, val_acc,  label='Validation Accuracy', linewidth = 3)
plt.title('Accuracy vs Epochs', fontsize = 25)
plt.xlabel('Epochs', fontsize = 15)
plt.ylabel('Accuracy', fontsize = 15)
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
model = load_model(MODEL_PATH)
valid_accuracy = round(accuracy_score(Y_valid, np.argmax(model.predict(X_valid), axis = -1)),2)
print("Valid accuracy:",valid_accuracy)